##                                                        Final Project
  
### Project Overview:
The purpose of the project is to see whether smalls businesses create more jobs in California. We use QWI (Quarterly Workforce Indicator), published by the US Census Bureau, as the main data source. Multiple linear regression models will be tested to better capture and isolate covariances between regressors.
#### (Important variables)
The depandent variable in the regression model will be the net job creation. This is calculated by subtracting number of hirings for replacement from the total number of hirings. 

The independent variable will be the proportion of employment of small firms with respect to the total employment in the county. This variable is to reflect the weighted proportion of small businesses. 

The first set of controlled variables will include the proportion of employments of medium sized and large firms. 

The second controlled variable is the education contribution level for each small firms. This is calculated by the number of employments with education level higher than or equal to undergraduate level divided by the total employments of small firms. 

The third controlled variable will be a dummy variable indicating the financial crisis from 2008 to 2010. 
### Project Structure:
#### (Introduction) 
The project will start with data description, which will include descriptive statistics for each variable. Also, the data structure of QWI will be briefly introduced since there are randoms distortions added for the confidentiality purpose. It is neccessary to consider this kind of error factors.
#### (Methodology) 
The second part will introduce main data process methodology including processing missing data and processing "significantly distorted value". Also, variables from the raw dataset will be processed to better eliminate factors that will affect the robustness of the regression, such as seasonality. After these steps, we can present the final processed dataset with simple descriptive statistics such as mean, variance, correlation and covariance. 
#### (Results) 
The third part will be constructing regression models. Both linear and logistic regression models will be tested with different regressor inputs. 
#### (Discussion) 
The fourth part will be checking the robustness of different regression models and the significance of important coefficients. Robustness testing will include the discussion of adjusted-R, covariance matrix and BIC so that the most preferred model could be selected. Also, the significance will be tested using t-test on important coefficients. 
#### (Conclusion) 
The final part will be interpreting the results in case of real-life suggestions and implications. Sugesstions for future research will be included as well.


